In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
from minijpas_LF_and_puricomp import effective_volume
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from scipy import linalg
from my_functions import schechter
from multiprocessing import Pool
from LAEs.my_functions import *
from LAEs.plot_puricomp2d import *

In [ ]:
%matplotlib inline

In [ ]:
def mask_puricomp(puri, comp, min_puri=0.2, min_comp=0.2):
    return (puri > min_puri) & (comp > min_comp)

min_N_bin = 0.0001

In [ ]:
def load_this():
    # My LF
    nbs_list = [[1, 5], [4, 8], [7, 11], [10, 14], [13, 17], [16, 20]]
    qso_factor = 1.0
    comb_nbs_list = nbs_list
    survey_list = [f'minijpasAEGIS00{i}' for i in range(1, 4 + 1)] + ['jnep']
    w_central = central_wavelength()
    w_lya = 1215.67
    nb_fwhm_Arr = nb_fwhm(np.arange(60))
    L_binning = np.load('npy/L_nb_err_binning.npy')
    b = np.log10(L_binning)
    LF_bins = np.array([(b[i] + b[i + 1]) / 2 for i in range(len(b) - 1)])
    bin_width = np.array([b[i + 1] - b[i] for i in range(len(b) - 1)])

    total_volume = 0.
    for [this_nb_min, this_nb_max] in comb_nbs_list:
        total_volume += effective_volume(this_nb_min, this_nb_max, 'both')
    masked_volume = None
    hist_mat = None
    LF_raw = None
    total_puri_list = np.load('npy/total_puri_list.npy')
    total_comp_list = np.load('npy/total_comp_list.npy')

    for i, [nb1, nb2] in enumerate(comb_nbs_list):
        pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
        comp_list, _, _, puri_list, comp_den_list, _, _, puri_den_list, puricomp_bins = \
            load_puricomp1d(pathname)

        # Bin centers
        bc = [puricomp_bins[i: i + 2].sum() * 0.5 for i in range(len(puricomp_bins) - 1)]

        this_puri = np.interp(LF_bins, bc, total_puri_list[i])
        this_comp = np.interp(LF_bins, bc, total_comp_list[i])
        this_hist = None
        for survey_name in survey_list:
            pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
            filename_hist = f'{pathname}/hist_i_mat_{survey_name}.npy'
            hist_i_mat = np.load(filename_hist)

            this_field_LF = hist_i_mat / effective_volume(nb1, nb2, survey_name) / bin_width

            if this_hist is None:
                this_hist = hist_i_mat
                field_LF_mat = this_field_LF
            else:
                this_hist += hist_i_mat
                field_LF_mat = np.vstack([field_LF_mat, this_field_LF])

        this_hist = this_hist / total_volume / bin_width

        this_volume = np.ones_like(LF_bins) * effective_volume(nb1, nb2, 'both')

        filename_dict = f'{pathname}/LFs.pkl'
        with open(filename_dict, 'rb') as file:
            this_LF_raw = pickle.load(file)['LF_total_raw'] * this_volume
            if LF_raw is None:
                LF_raw = this_LF_raw
            else:
                LF_raw += this_LF_raw

        # Set masked bins by puricomp_mask to 0
        N_median_hist = np.nanmedian(this_hist, axis=0) * total_volume * bin_width
        puricomp_mask = mask_puricomp(this_puri, this_comp) & (N_median_hist >= min_N_bin)
        this_hist[:, ~puricomp_mask] = 0.
        this_volume[~puricomp_mask] = 0.

        if masked_volume is None:
            masked_volume = this_volume
        else:
            masked_volume += this_volume

        if hist_mat is None:
            hist_mat = this_hist
        else:
            hist_mat = hist_mat + this_hist

    hist_mat = hist_mat * total_volume / masked_volume

    # Field by field variation
    field_L_LF_err_percentiles = np.nanpercentile(field_LF_mat, [16, 50, 84], axis=0)
    field_LF_err_plus = field_L_LF_err_percentiles[2] - field_L_LF_err_percentiles[1]
    field_LF_err_minus = field_L_LF_err_percentiles[1] - field_L_LF_err_percentiles[0]

    L_LF_err_percentiles = np.nanpercentile(hist_mat, [16, 50, 84], axis=0)
    LF_err_plus = L_LF_err_percentiles[2] - L_LF_err_percentiles[1]
    LF_err_minus = L_LF_err_percentiles[1] - L_LF_err_percentiles[0]
    hist_median = L_LF_err_percentiles[1]

    mask_low_N = (LF_raw < min_N_bin)
    for h in [LF_err_plus, LF_err_minus, hist_median]:
        h[mask_low_N] = 0.

    volwid = total_volume * bin_width

    load_LF_name = '/home/alberto/cosmos/LAEs/Luminosity_functions/Total_LF'

    yerr_plus = np.load(f'{load_LF_name}/LF_err_plus.npy')
    yerr_minus = np.load(f'{load_LF_name}/LF_err_minus.npy')

    LF_boots = np.load(f'{load_LF_name}/median_LF_boots.npy')
    # Fix yerr_minus:
    yerr_minus[LF_boots == 0] = hist_median[LF_boots == 0]


    LF_dict = {
        'LF_bins': LF_bins,
        'LF_total': hist_median,
        'LF_total_err': [yerr_minus, yerr_plus]
    }
    
    return LF_dict

LF_dict = load_this()

LF_yerr_minus = LF_dict['LF_total_err'][0]
LF_yerr_plus = LF_dict['LF_total_err'][1]
LF_phi = LF_dict['LF_total']
LF_bins = LF_dict['LF_bins']

In [ ]:
def covmat_simple(xiallreal):

    nbins, nreals = xiallreal.shape

    ximeans = xiallreal.mean(axis=1)
    xistds = xiallreal.std(axis=1, ddof=1)
    covmat = np.empty((nbins, nbins), float)
    corrmat = np.empty((nbins, nbins), float)

    for i in range(nbins):
        for j in range(nbins):
            covmat[i, j] = ((xiallreal[i] - ximeans[i]) *
                            (xiallreal[j] - ximeans[j])).sum()/float(nreals - 1)
            corrmat[i, j] = covmat[i, j]/(xistds[i]*xistds[j])

    return covmat, corrmat


def compute_invcovmat(hist_mat, where_fit):
    this_hist_mat = hist_mat.T[where_fit]
    this_hist_mat[~np.isfinite(this_hist_mat)] = 0.
    covmat = covmat_simple(this_hist_mat)[0]
    invcovmat = linalg.inv(covmat)
    return invcovmat, covmat

In [ ]:
# The fitting surve
def power_fit(Lx, A, B):
    return 10 ** (A * (np.log10(Lx) - 43.5) + B)

# The fitting curve used in literature
def ref_power_fit(Lx, A, B):
    return 10 ** (A * (np.log10(Lx)) + B)

In [ ]:
def chi2_fullmatrix(data_vals, inv_covmat, model_predictions):
    """
    Given a set of data points, its inverse covariance matrix and the
    corresponding set of model predictions, computes the standard chi^2
    statistic (using the full covariances)
    """

    y_diff = data_vals - model_predictions
    return np.dot(y_diff, np.dot(inv_covmat, y_diff))

def prior_f(theta):
    A0 = theta[0]
    B0 = theta[1]
    A_range = (-2.5 < A0 < 1)
    B_range = (-7 < B0 < -3)

    if A_range & B_range:
        return 0.
    else:
        return -np.inf

def log_likelihood(theta, Lx, Phi, invcovmat):
    A0 = theta[0]
    B0 = theta[1]

    model_Arr = power_fit(Lx, A0, B0)
    
    chi2 = chi2_fullmatrix(model_Arr, invcovmat, Phi)

    return -0.5 * chi2

def log_p(theta, Lx, Phi, invcovmat):
    return log_likelihood(theta, Lx, Phi, invcovmat) + prior_f(theta)

In [ ]:
def load_and_compute_invcovmat(where_fit):
    # Load the matrix of LF realizations
    pathname = f'Luminosity_functions/Total_LF'
    filename_hist = f'{pathname}/hist_mat.npy'
    hist_i_mat = np.load(filename_hist)
    return compute_invcovmat(hist_i_mat, where_fit)

In [ ]:
## MCMC parameters ##
N_walkers = 500
N_burn = 100_000
N_steps = 10_000

# Error to use
yerr = (LF_yerr_plus + LF_yerr_minus) * 0.5
yerr[LF_phi == 0] = np.inf

# In which LF bins fit
where_fit = np.isfinite(yerr) & (LF_bins > 40) & (LF_bins < 44.5) & (yerr > 0)

invcovmat = load_and_compute_invcovmat(where_fit)[0]

theta0 = np.zeros((N_walkers, 2))
theta0[:, 0] = np.random.normal(-1.07, 1e-4, N_walkers)
theta0[:, 1] = np.random.normal(-5.05, 1e-4, N_walkers)

args = (10**LF_bins[where_fit], LF_phi[where_fit], invcovmat)

with Pool() as pool:
    # Ensemble the sampler
    sampler = emcee.EnsembleSampler(N_walkers, 2, log_p, args=args, pool=pool)
    # Run the MCMC
    state = sampler.run_mcmc(theta0, N_burn, progress=True, thin=1);
    sampler.reset()
#     state = sampler.run_mcmc(state, N_burn, progress=True, thin=10);
#     sampler.reset()
    sampler.run_mcmc(state, N_steps, progress=True, thin=1)

# autocorr_t = sampler.get_autocorr_time()
# print(f'Autocorrelation time: {autocorr_t}')

In [ ]:
flat_samples = sampler.get_chain(discard=N_steps // 5 * 4, thin=5, flat=True)
labels = ['A', 'B', 'log_f']

[A_fit, B_fit] = np.median(flat_samples, axis=0)
[A_perc84, B_perc84] = np.percentile(flat_samples, [84], axis=0)[0]
[A_perc16, B_perc16] = np.percentile(flat_samples, [16], axis=0)[0]

A_fit_err_up = A_perc84 - A_fit
B_fit_err_up = B_perc84 - B_fit
A_fit_err_down = A_fit - A_perc16
B_fit_err_down = B_fit - B_perc16

print(f'Best fit:\n'
      f'A = {A_fit:0.2f} + {A_fit_err_up:0.2f} - {A_fit_err_down:0.2},\n'
      f'B = {B_fit:0.2f} + {B_fit_err_up:0.2f} - {B_fit_err_down:0.2}')

fig = corner.corner(flat_samples, labels=labels,
                    truths=[A_fit, B_fit])
plt.show(block=False)

# Save the chain
np.save('npy/mcmc_powerlaw_fit_chain', flat_samples)

In [ ]:
from my_functions import double_schechter
fig, ax = plt.subplots(figsize=(6, 4))

# Compute fit for all steps in chain
Phi_fit_i = []
Lx = np.logspace(42, 46, 1000)
for ii, step in enumerate(flat_samples[::-1]):
    if ii == 10_000:
        break
    Phi_fit_i.append(power_fit(Lx, step[0], step[1]))
Phi_fit_84 = np.percentile(Phi_fit_i, 84, axis=0)
Phi_fit_16 = np.percentile(Phi_fit_i, 16, axis=0)

# My fit
Phi_fit = power_fit(Lx, A_fit, B_fit)
ax.plot(np.log10(Lx), Phi_fit, lw=2, label='Best fit (This work)')
ax.fill_between(np.log10(Lx), Phi_fit_16, Phi_fit_84, alpha=0.5,
                color='C0', lw=0)

# Sobral 2018
Lxx = np.logspace(42, 46, 1000)
s18_A_fit = -0.75
s18_B_fit = 27.1
Phi_fit = ref_power_fit(Lxx, s18_A_fit, s18_B_fit)
ax.plot(np.log10(Lxx), Phi_fit,
        ls=(5, (10, 3)), zorder=96, lw=2,
        label='Sobral 2018')
# Sobral 2017
Lxx = np.logspace(42, 46, 1000)
s17_A_fit = -1.48
s17_B_fit = 59.4
Phi_fit = ref_power_fit(Lxx, s17_A_fit, s17_B_fit)
ax.plot(np.log10(Lxx), Phi_fit,
        ls='--', zorder=96, lw=2,
        label='Sobral 2017')
# Matthee 2017
Lxx = np.logspace(42, 46, 1000)
m17_A_fit = -0.74
m17_B_fit = 27.5
Phi_fit = ref_power_fit(Lxx, m17_A_fit, m17_B_fit)
ax.plot(np.log10(Lxx), Phi_fit,
        ls='-.', zorder=96, lw=2,
        label='Matthee 2017')


ax.errorbar(LF_bins[where_fit], LF_phi[where_fit],
            yerr=[LF_yerr_minus[where_fit], LF_yerr_plus[where_fit]],
            fmt='s', mfc='r', mec='k', ecolor='k', zorder=999,
            mew=1.5, ms=7, capsize=4, label='This work ($z=2.05$-$3.75$)')
ax.errorbar(LF_bins[~where_fit], LF_phi[~where_fit],
            yerr=[LF_yerr_minus[~where_fit], LF_yerr_plus[~where_fit]],
            fmt='s', mfc='none', mec='k', ecolor='k', zorder=999,
            mew=1.5, ms=7, capsize=4)

ax.set_yscale('log')
ax.set_ylim(1e-8, 5e-3)
ax.set_xlim(42.5, 45.5)

# Plot the reference LF curves

ax.set_ylim(1e-8, 1e-2)
ax.set_xlim(42.5, 45.5)
ax.set_yscale('log')

ax.set_xlabel(r'$\log_{10}L_{\mathrm{Ly}\alpha}$ [erg$\,$s$^{-1}$]', fontsize=18)
ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\,\Delta\log_{10}L^{-1}$]', fontsize=18)

ax.tick_params(labelsize=18, direction='in', which='both')
ax.set_yticks(np.logspace(-8, -2, 7))
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.legend(fontsize=11)

fig.savefig('figures/Powerlaw_fit.pdf', bbox_inches=None, pad_inches=0.1, facecolor='w')
plt.show(block=False)